<b> Modeling </b> notebook (number 3) for <b>Project 2</b> of General Assembly DSIR-0320 <br>
by Martijn de Vries </br>
martijndevries91@gmail.com

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

#sklearn imports
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error